# Project + Community Matchup Data ETL

Pipeline for processing responses collected from a Google Form survey into data formatted for statistical analysis.

Intended data usage is to create:
- A matchup chart
- A confidence rating for each matchup
- Tier lists based on multiple statistical factors

## Data import from survey submissions 📓

Pull the data from the csv and strip out the uneccessary columns, leaving a dataframe with only characters, player skill, and matchup data.

%pip install pandas

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('P+ Community Matchup Chart Form Responses.csv')

df.rename(columns={'Please select a character to submit matchup data for.': 'Character', 'What is your approximate skill level?' : 'Skill', 'What is your tag?': 'Tag', 'I attest that the data I am submitting is my actual opinion, and I am not submitting junk data to cause a headache.' : 'Attestation'}, inplace=True)

matchupDf = df.drop(columns=['Timestamp', 'Tag','Attestation'])
matchupDf = matchupDf.sort_values(by=['Character', 'Skill'])

print(matchupDf.head(5))

          Character                                              Skill Bowser  \
179          Bowser  High Level (I regularly make it out of pools i...  50:50   
42           Bowser  Intermediate (I have participated in multiple ...  50:50   
121          Bowser  Intermediate (I have participated in multiple ...  50:50   
163          Bowser  Top Player (I have won regional or larger size...  50:50   
34   Captain Falcon  High Level (I regularly make it out of pools i...  70:30   

    Captain Falcon Charizard Diddy Kong Donkey Kong  Falco    Fox Ganondorf  \
179          50:50     55:45      45:55       50:50  60:40  60:40     60:40   
42           30:70     45:55        NaN       55:45  40:60  50:50     45:55   
121          60:40     70:30      40:60       50:50  30:70  70:30     60:40   
163          40:60     40:60      55:45       40:60  60:40  60:40     40:60   
34           50:50     70:30      55:45       60:40  45:55  30:70     70:30   

     ...  Sheik  Snake  Sonic Squirtle

## Transform data 🔄

Transform the raw data to a format suitable for statistical analysis.
- Matchup ratios converted to a numerical value ranging from -3 to 3
- Skill levels converted to a weight between 1 and 5

In [2]:
matchupDf = matchupDf.replace({'50:50': 0,'55:45': .5,'60:40': 1,'70:30': 2,'80:20': 3,'45:55': -.5,'40:60': -1,'30:70': -2,'20:80': -3}, regex=True)

matchupDf = matchupDf.replace({'Intermediate \(I have participated in multiple tournaments\)': 1,'High Level \(I regularly make it out of pools in tournaments\)': 3,'Top Player \(I have won regional or larger sized tournaments\)': 5}, regex=True)

print(matchupDf.head(5))

          Character  Skill  Bowser  Captain Falcon  Charizard  Diddy Kong  \
179          Bowser      3     0.0             0.0        0.5        -0.5   
42           Bowser      1     0.0            -2.0       -0.5         NaN   
121          Bowser      1     0.0             1.0        2.0        -1.0   
163          Bowser      5     0.0            -1.0       -1.0         0.5   
34   Captain Falcon      3     2.0             0.0        2.0         0.5   

     Donkey Kong  Falco  Fox  Ganondorf  ...  Sheik  Snake  Sonic  Squirtle  \
179          0.0    1.0  1.0        1.0  ...   -1.0    0.5   -0.5       0.0   
42           0.5   -1.0  0.0       -0.5  ...   -2.0   -1.0   -1.0      -1.0   
121          0.0   -2.0  2.0        1.0  ...   -1.0   -3.0   -2.0      -0.5   
163         -1.0    1.0  1.0       -1.0  ...   -0.5    0.5   -1.0      -1.0   
34           1.0   -0.5 -2.0        2.0  ...   -2.0    0.5    2.0       0.5   

     Toon Link  Wario  Wolf  Yoshi  Zelda  Zero Suit Samus  
1

### Data weighting 🏋️‍♂️

Duplicate rows based on skill level to reflect the weight of each entry.<br>
While not an efficient solution if considering memory constraints, does make later analysis more straightforward.

In [3]:
expandedDf = matchupDf.loc[matchupDf.index.repeat(matchupDf.Skill)]

expandedDf = expandedDf.drop(columns=['Skill'])

print(expandedDf)

           Character  Bowser  Captain Falcon  Charizard  Diddy Kong  \
179           Bowser     0.0             0.0        0.5        -0.5   
179           Bowser     0.0             0.0        0.5        -0.5   
179           Bowser     0.0             0.0        0.5        -0.5   
42            Bowser     0.0            -2.0       -0.5         NaN   
121           Bowser     0.0             1.0        2.0        -1.0   
..               ...     ...             ...        ...         ...   
19   Zero Suit Samus     1.0             0.5        0.5         1.0   
19   Zero Suit Samus     1.0             0.5        0.5         1.0   
19   Zero Suit Samus     1.0             0.5        0.5         1.0   
19   Zero Suit Samus     1.0             0.5        0.5         1.0   
19   Zero Suit Samus     1.0             0.5        0.5         1.0   

     Donkey Kong  Falco  Fox  Ganondorf  Ice Climbers  ...  Sheik  Snake  \
179          0.0    1.0  1.0        1.0           0.0  ...   -1.0    0.

### Aggregation 🙏

Aggregate row data into lists.
- Each row and column represent a character
- Each data cell contains a list of all matchup data submitted for a given combination of characters

In [4]:
aggregatedDf = expandedDf.groupby('Character').agg(lambda x: list(x.dropna()))

print(aggregatedDf.head(3))

                                                           Bowser  \
Character                                                           
Bowser          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
Captain Falcon  [2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, ...   
Charizard       [0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...   

                                                   Captain Falcon  \
Character                                                           
Bowser          [0.0, 0.0, 0.0, -2.0, 1.0, -1.0, -1.0, -1.0, -...   
Captain Falcon  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
Charizard       [0.0, 0.0, 0.0, -0.5, -0.5, -0.5, 1.0, 1.0, 1....   

                                                        Charizard  \
Character                                                           
Bowser          [0.5, 0.5, 0.5, -0.5, 2.0, -1.0, -1.0, -1.0, -...   
Captain Falcon  [2.0, 2.0, 2.0, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, ...   
Charizard       [0.0, 0.0, 0.0, 

### Duplication 👥
Matchup data is propagated from each cell to it's corresponding inverse cell.<br>
*e.g. the data from the Bowser vs Captain Falcon cell is duplicated, negated, and appended to the data in the Captain Falcon vs Bowser cell*

In [5]:
combinedDf = aggregatedDf.copy()

for character in aggregatedDf.index:
    for opponent in aggregatedDf.columns:
        combinedList = aggregatedDf[opponent][character] + [i * -1 for i in (aggregatedDf[character][opponent])]
        combinedDf[opponent][character] = combinedList

We can now access all submitted matchup data between any two charactres with a lookup of their corresponding column and row.<br>
For example, here is all the data in the Captain Falcon vs Marth matchup from the perspective of the Captain Falcon players:

In [6]:
print(combinedDf["Marth"]["Captain Falcon"])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, -0.0, -0.0, -0.0, 0.5, 0.5, 2.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.5, -0.5, -0.5, -0.5, -0.5, -0.0, -0.0, -0.0, -0.0, -0.0, -0.5, -0.5, -0.5, -0.5, -0.5]


And here is the data from the Marth player perspective:

In [7]:
print(combinedDf["Captain Falcon"]["Marth"])

[0.0, 0.0, 0.0, -0.5, -0.5, -2.0, 0.0, -1.0, -1.0, -1.0, -1.0, -1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.5, -0.5, -0.5, -0.5, -0.5]


## Calculations 📊

Calculate the mean and standard error for each matchup.

In [8]:
#Mean
def mean(data):
    return sum(data) / len(data)

meanDf = combinedDf.applymap(mean)

print(meanDf.head(5))

                  Bowser  Captain Falcon  Charizard  Diddy Kong  Donkey Kong  \
Character                                                                      
Bowser          0.000000       -1.280000  -0.634615   -0.666667    -0.666667   
Captain Falcon  1.280000        0.000000   0.176471   -0.189655    -0.019231   
Charizard       0.634615       -0.176471   0.000000   -0.393939     0.283333   
Diddy Kong      0.666667        0.189655   0.393939    0.000000    -0.680000   
Donkey Kong     0.666667        0.019231  -0.283333    0.680000     0.000000   

                   Falco       Fox  Ganondorf  Ice Climbers       Ike  ...  \
Character                                                              ...   
Bowser         -0.175000  0.325000  -0.648148     -1.395833 -0.774194  ...   
Captain Falcon -0.714286 -0.576923   0.859375      0.948276 -0.040541  ...   
Charizard      -0.062500  0.071429   0.857143     -1.440000  0.083333  ...   
Diddy Kong      0.104167 -0.250000   0.467742    

In [9]:
#Standard error of the mean
def sem(data):
    return np.std(data, ddof=1) / np.sqrt(np.size(data))

semDf = combinedDf.applymap(sem)

print(semDf.head(5))

                  Bowser  Captain Falcon  Charizard  Diddy Kong  Donkey Kong  \
Character                                                                      
Bowser          0.000000        0.168523   0.142930    0.206116     0.159364   
Captain Falcon  0.168523        0.000000   0.163357    0.127533     0.142723   
Charizard       0.142930        0.163357   0.000000    0.089076     0.119104   
Diddy Kong      0.206116        0.127533   0.089076    0.000000     0.099499   
Donkey Kong     0.159364        0.142723   0.119104    0.099499     0.000000   

                   Falco       Fox  Ganondorf  Ice Climbers       Ike  ...  \
Character                                                              ...   
Bowser          0.230203  0.233043   0.152880      0.162016  0.083081  ...   
Captain Falcon  0.094940  0.163360   0.087628      0.189543  0.095661  ...   
Charizard       0.223835  0.155827   0.109566      0.101325  0.083333  ...   
Diddy Kong      0.162016  0.214932   0.100898    

## Export ➡️

Save the data to csv. Will import the data in Google Sheets to format and share results.

In [13]:
meanDf.to_csv('mean_data.csv')
semDf.to_csv('sem_data.csv')